In [5]:
from rich import print
from sqlalchemy.orm import DeclarativeBase, Mapped, mapped_column
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy import String

In [2]:
class Base(DeclarativeBase):
    id: Mapped[int] = mapped_column(primary_key=True, autoincrement=True)

In [ ]:
class User(Base):
    __tablename__ = 'users'
    login: Mapped[str] = mapped_column(String(20), unique=True)
    name: Mapped[str] = mapped_column(String(50))


print(f'{User.__table__!r}')

Table('users', MetaData(), Column('login', String(length=20), table=<users>, nullable=False), Column('name', 
String(length=50), table=<users>, nullable=False), Column('id', Integer(), table=<users>, primary_key=True, 
nullable=False), schema=None)

In [ ]:
engine = create_engine('sqlite:///:memory:')

Base.metadata.create_all(engine)

## Insert data

In [8]:
with Session(engine) as session:
    users = [
        User(login='alex', name='Alex Petrov'),
        User(login='ivan', name='Ivan Semenov'),
    ]

    session.add_all(users)

    session.commit()

## Select data

In [10]:
with Session(engine) as session:
    for user in session.query(User).order_by(User.id):
        print(f'{user.login}:  {user.name}')

alex:  Alex Petrov

ivan:  Ivan Semenov

## Delete data

In [ ]:
with Session(engine) as session:
    user = session.get(User, 1)
    session.delete(user)
    session.commit()

    print(f'User: {user.login} deleted')

User: alex deleted

## Update data

In [12]:
with Session(engine) as session:
    user = session.get(User, 2)
    user.name = 'Ivan Ivanov'
    session.commit()

    print(f'User: {user.login} updated')

User: ivan updated

In [20]:
with Session(engine) as session:
    users = session.query(User)

    for user in enumerate(users):
        print(f'{user[0]}: {user[1].login} - {user[1].name}')

0: ivan - Ivan Ivanov